In [4]:
import time
import pandas as pd
import paho.mqtt.client as mqtt

BROKER = "broker.hivemq.com"
PORT = 1883
TOPIC_SETPOINT = "imclab/motor/setpoint"
TOPIC_SPEED = "imclab/motor/speed"
TOPIC_LED = "imclab/led"

log = []
speed_now = 0
setpoint_now = 0

def on_connect(client, userdata, flags, rc):
    print(f"✓ Connected to broker (rc={rc})")
    client.subscribe(TOPIC_SPEED)
    print("✓ Subscribed to motor speed feedback\n")

def on_message(client, userdata, msg):
    global speed_now
    if msg.topic == TOPIC_SPEED:
        speed_now = int(msg.payload.decode())
        log.append({
            "time": time.time(),
            "setpoint": setpoint_now,
            "speed": speed_now,
            "error": setpoint_now - speed_now
        })

c = mqtt.Client()
c.on_connect = on_connect
c.on_message = on_message
c.connect(BROKER, PORT, 60)
c.loop_start()

def set_motor(val):
    global setpoint_now
    setpoint_now = int(val)
    c.publish(TOPIC_SETPOINT, str(setpoint_now))
    print(f"→ Setpoint: {setpoint_now}")

def set_led(state):
    c.publish(TOPIC_LED, "1" if state else "0")
    print(f"→ LED: {'ON' if state else 'OFF'}")

# === STEP TEST ===
print("="*60)
print("       MOTOR EXPERIMENT - DATA COLLECTION")
print("="*60)
print("\n⚠️  Jangan ganggu ESP32 selama ~25 detik...\n")

time.sleep(2)  # tunggu subscribe ready

set_led(True)
time.sleep(1)

set_motor(0); time.sleep(3)
set_motor(100); time.sleep(4)
set_motor(180); time.sleep(4)
set_motor(255); time.sleep(4)
set_motor(150); time.sleep(3)
set_motor(80); time.sleep(3)
set_motor(0); time.sleep(2)

set_led(False)

print("\n" + "="*60)
print("       COLLECTION DONE - SAVING DATA...")
print("="*60 + "\n")

time.sleep(1)
c.loop_stop()
c.disconnect()

# Save CSV
df = pd.DataFrame(log)

if len(df) == 0:
    print("❌ ERROR: Tidak ada data yang terkumpul!")
    print("\nCek:")
    print("  1. ESP32 Serial Monitor muncul 'MQTT...OK'?")
    print("  2. WiFi ESP32 connected?")
    print("  3. Motor respond saat test manual dari HP?")
else:
    df.to_csv("motor_response_data.csv", index=False)
    print(f"✓ Saved {len(df)} samples to motor_response_data.csv")
    print(f"\nData info:")
    print(f"  - Columns: {list(df.columns)}")
    print(f"  - Shape: {df.shape}")
    print(f"  - Mean error: {df['error'].mean():.2f}")
    print(f"  - Std error: {df['error'].std():.2f}")
    
    print("\n" + "="*60)
    print("   DATA COLLECTION COMPLETE! 🎉")
    print("="*60)
    print("\n📂 File created: motor_response_data.csv")
    print("🚀 Ready for PROJECT 2 (LSTM)!")
    print("="*60 + "\n")
    
    print("Preview 10 baris pertama:")
    display(df.head(10))


C:\Users\RAKA\AppData\Local\Temp\ipykernel_24848\3280664943.py:31: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  c = mqtt.Client()


       MOTOR EXPERIMENT - DATA COLLECTION

⚠️  Jangan ganggu ESP32 selama ~25 detik...

→ LED: ON
→ Setpoint: 0
✓ Connected to broker (rc=0)
✓ Subscribed to motor speed feedback

→ Setpoint: 100
→ Setpoint: 180
→ Setpoint: 255
→ Setpoint: 150
→ Setpoint: 80
→ Setpoint: 0
→ LED: OFF

       COLLECTION DONE - SAVING DATA...

✓ Saved 46 samples to motor_response_data.csv

Data info:
  - Columns: ['time', 'setpoint', 'speed', 'error']
  - Shape: (46, 4)
  - Mean error: 0.00
  - Std error: 31.39

   DATA COLLECTION COMPLETE! 🎉

📂 File created: motor_response_data.csv
🚀 Ready for PROJECT 2 (LSTM)!

Preview 10 baris pertama:


,time,setpoint,speed,error
0,1.766131e+09,0,0,0
1,1.766131e+09,0,0,0
2,1.766131e+09,0,0,0
3,1.766131e+09,100,0,100
4,1.766131e+09,100,100,0
5,1.766131e+09,100,100,0
6,1.766131e+09,100,100,0
7,1.766131e+09,100,100,0
8,1.766131e+09,100,100,0
9,1.766131e+09,100,100,0
